In [ ]:
# NEO4J_URI=bolt://localhost:7687
# NEO4J_USERNAME=neo4j
# NEO4J_PASSWORD=Admin@123
# NEO4J_DATABASE=neo4j

# OPENAI_API_KEY=<api-key>

In [ ]:
# pip install
# langchain
# langchain-community
# sentence-transformers
# langchain-experimental
# neo4j
# pypdf
# python-dotenv
# langchain-openai
# streamlit

In [5]:
from neo4j import GraphDatabase
from typing import Union
import os
from dotenv import load_dotenv

load_dotenv()

True

In [26]:
class Neo4jConnection:
    def __init__(self, uri, username, password, database):
        self.uri = uri
        self.username = username
        self.password = password
        self.database = database
        self.driver = None
        
        self.driver = GraphDatabase.driver(self.uri, auth=(self.username, self.password))
        print("Connection to Neo4j DB successful")
        
    def query(self, query, parameters={}):
        assert self.driver is not None, "Driver is not initialized"
        
        try:
            session = self.driver.session(database=self.database)
            response = list(session.run(query, parameters))
            
        except Exception as e:
            print(f"query failed: {e}")
            
        return response
            
    def write_transaction(self, query, parameters={}):
        assert self.driver is not None, "Driver not initialized"
        session = None
        
        try:
            session = self.driver .session(database= self.database)
            session.execute_write(lambda tx: tx.run(query, parameters or {}))
            
        except Exception as e:
            print(f"Write transaction failed: {e}")

In [27]:
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
database = os.getenv("NEO4J_DATBASE", "neo4j")

conn = Neo4jConnection(uri, username, password, database)

try:
    test_query = "MATCH (n) RETURN count(n) as node_count"
    result = conn.query(test_query)
    
    if result:
        print(f"Total nodes in the database: {result[0]['node_count']}")
        
    create_query = """
    CREATE (p:Person {name: 'Test User', age: 28})
    RETURN p
    """
    
    conn.write_transaction(create_query)
    print("created test node succesfully")
    
    verify_query = "MATCH (p:Person {name: 'Test User'}) RETURN p"
    result = conn.query(verify_query)
    
    if result:
        print(f"Test node found, {dict(result[0]['p'])}")
    
except Exception as e:
    print(f"query failed: {e}")
    

Connection to Neo4j DB successful
Total nodes in the database: 4
created test node succesfully
Test node found, {'name': 'Test User', 'age': 28}
